In [28]:
import pandas as pd
import numpy as 
import os

his_path = 'sim/2030/logs/his'
data_files = [f for f in os.listdir(his_path) if os.path.isfile(os.path.join(his_path, f))]
data_files = [f for f in data_files if '.bak' not in f]
data_files

['host_0_tracker_0.csv', 'host_0_tracker_1.csv']

In [16]:
RES_CONSUMP = 0.00131 # MW

In [3]:
df = pd.DataFrame()
for f in data_files:
    df = pd.concat([df, pd.read_csv(os.path.join(his_path, f))])
print("results from: {} generators".format(len(df.id.unique())))
df

results from: 18 generators


,id,ts,host,tracker,output,demand,net_cap,net_cap_percent
0,-195991192,1,0,0,118.601973,105.00,13.601973,0.114686
1,-1293263167,1,0,0,118.601973,105.00,13.601973,0.114686
2,-728242845,1,0,0,28.920975,42.75,-13.829025,-0.478166
3,-1675485352,1,0,0,88.200000,73.50,14.700000,0.166667
4,-1719963416,1,0,0,118.601973,105.00,13.601973,0.114686
...,...,...,...,...,...,...,...,...
375,1447796223,38,0,1,120.092099,105.00,15.092099,0.125671
376,991293890,38,0,1,88.200000,73.50,14.700000,0.166667
377,844283200,38,0,1,120.092099,105.00,15.092099,0.125671
378,1811534858,38,0,1,32.624891,42.00,-9.375109,-0.287361


Find instances where capacity was negative

In [4]:
df.groupby(['ts','tracker']).first().head()

id  host      output  demand    net_cap  net_cap_percent
ts tracker                                                                  
1  0        -195991192     0  118.601973   105.0  13.601973         0.114686
   1         743607863     0   88.200000    73.5  14.700000         0.166667
2  0        -195991192     0  116.224898   105.0  11.224898         0.096579
   1        1447796223     0  116.224898   105.0  11.224898         0.096579
3  0        -195991192     0  112.216429   105.0   7.216429         0.064308

In [5]:
t = df[(df.ts == 5) & (df.tracker == 1)]
t

,id,ts,host,tracker,output,demand,net_cap,net_cap_percent
40,1367406317,5,0,1,877.500000,731.25,146.250000,0.166667
41,743607863,5,0,1,88.200000,73.50,14.700000,0.166667
42,1447796223,5,0,1,118.956372,105.00,13.956372,0.117323
43,1043607079,5,0,1,88.200000,73.50,14.700000,0.166667
44,1283859926,5,0,1,118.956372,105.00,13.956372,0.117323
45,1863679415,5,0,1,118.956372,105.00,13.956372,0.117323
46,991293890,5,0,1,88.200000,73.50,14.700000,0.166667
47,1811534858,5,0,1,30.141272,42.00,-11.858728,-0.393438
48,1943371260,5,0,1,88.200000,73.50,14.700000,0.166667
49,844283200,5,0,1,118.956372,105.00,13.956372,0.117323


In [6]:
t = df[df.ts == 5].groupby(['tracker']).get_group(1).set_index('id')
t.T

id,1367406317,743607863,1447796223,1043607079,1283859926,1863679415,991293890,1811534858,1943371260,844283200
ts,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
host,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
tracker,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
output,877.500000,88.200000,118.956372,88.200000,118.956372,118.956372,88.200000,30.141272,88.200000,118.956372
demand,731.250000,73.500000,105.000000,73.500000,105.000000,105.000000,73.500000,42.000000,73.500000,105.000000
net_cap,146.250000,14.700000,13.956372,14.700000,13.956372,13.956372,14.700000,-11.858728,14.700000,13.956372
net_cap_percent,0.166667,0.166667,0.117323,0.166667,0.117323,0.117323,0.166667,-0.393438,0.166667,0.117323


find time stamps where net_cap was negative.

In [25]:
deficit = t[t.net_cap_percent < 0.15]
surplus = t[t.net_cap_percent > 0]

load_to_shift = deficit.net_cap.sum()
print('need to shift {:.1f} MW'.format(load_to_shift))
print('{:,} homes'.format(np.ceil(load_to_shift / RES_CONSUMP).astype(int)))

available_capacity = surplus.net_cap.sum()
print('able to shift {:.1f} MW'.format(available_capacity))

In [8]:
ts = 5
net_system_cap = supply.net_cap.sum() - deficit.net_cap.sum()
print("at ts {} available capacity was {:.3f} MW".format(str(ts).rjust(3,"0"), net_system_cap))

at ts 005 available capacity was 216.909 MW


Running the below code reveals that each tracker always contains spare capacity

# Changed net_cap_percentto percent_use

In [337]:
df.columns = ['id', 'ts', 'host', 'tracker', 'output', 'demand', 'net_cap', 'percent_use']
df.index.name = 'e_id'
df.output = np.round(df.output, 3)
safety_threshold = 0.1
for tracker in range(df.tracker.max() + 1):
    print("Tracker {}".format(tracker))
    print("===========")
    for ts in range(df.ts.max())[-10:]:
        _ = df[(df.ts == ts) & (df.tracker == tracker)].copy()
        _['s_demand'] = np.round((_.output * (1 - safety_threshold)),2)
        _['s_net_cap'] =  _['s_demand'] - _['demand']
        deficit = _[_['percent_use'] < safety_threshold].copy()
        surplus = _[_['percent_use'] > safety_threshold].copy()
        net_system_cap = supply.net_cap.sum() - deficit.net_cap.sum()
        total_available_capacity = (surplus.output * (1 - safety_threshold) - surplus.demand).sum()

        print("at ts = {}".format(str(ts).rjust(2,"0")))
#         print(_[['output','demand','s_demand','net_cap','s_net_cap','percent_use']])
        print("------------------------------------")
        print("   - total capacity was  {:.2f} MW".format(_.output.sum()))
        print("   - total demand was    {:.2f} MW".format(_.demand.sum()))  
        print("                         ---------")
        print("   - spare capacity was  {:.2f} MW    // ignoring safety factor".format(net_system_cap))
        print("------------------------------------")
        print('   - able to shift       {} MW    // considering safety factor'.format('{:.1f}'.format(total_available_capacity).rjust(6)))
        print("------------------------------------")
        safe_load   = (_.output * (1 - safety_threshold)).sum()
        unsafe_load = (deficit.output * (1 - safety_threshold) - deficit.demand).sum()
        load_to_shift = safe_load - deficit.demand.sum()
        print("   - 'Safe Load' was     {:.2f} MW    // considering safety factor".format(safe_load))

        if unsafe_load < 0:
        # else deficit.output - deficit.demand = (-) load must be shifted
            print("   - 'Unsafe Load' was   {} MW    // need to shift".format('({:.1f})'.format(abs(unsafe_load)).rjust(6, " ")))
            homes_to_shift = np.ceil(np.abs(unsafe_load) / RES_CONSUMP).astype(int)
            surplus['s_net_cap'] = (surplus.output * (1 - safety_threshold)) - surplus.demand
            deficit['s_net_cap'] = (deficit.output * (1 - safety_threshold)) - deficit.demand
            surplus['share_of_net'] =  surplus['s_net_cap'] / surplus['s_net_cap'].sum()
            deficit['share_of_net'] =  deficit['s_net_cap'] / deficit['s_net_cap'].sum()
            surplus['new_load'] = -1 * np.round((surplus['share_of_net'] * unsafe_load) / RES_CONSUMP) * RES_CONSUMP
            deficit['new_load'] = -1 * surplus['new_load'].sum() * deficit['share_of_net']
            s = surplus['percent_use'].sort_values(ascending=True)

            print('   - Load shifting to   {} homes // {:,.1f} MW shifted'.format('{:,}'.format(homes_to_shift).rjust(7),
                                                                                  abs(surplus.new_load.sum()),abs(surplus.new_load.sum())))
            # 38 char
#             print('{}// {:,.1f} MW shifted'.format("".rjust(38), ))
            print("------------------------------------")
            print("All Data")
            print(pd.concat([surplus,deficit])[['output','demand','net_cap','s_net_cap','share_of_net','new_load']])
        else:
            # if   deficit.output - deficit.demand = (+) then no load need be shifted
            print("   - 'Unsafe Load' was        0 MW    ".format(load_to_shift)) # // {:.2f}
            print("------------------------------------")
            print("NO LOAD TO SHIFT".center(36))
        print("=" * 36)

Tracker 0
at ts = 28
------------------------------------
   - total capacity was  618.38 MW
   - total demand was    620.25 MW
                         ---------
   - spare capacity was  321.58 MW    // ignoring safety factor
------------------------------------
   - able to shift         17.6 MW    // considering safety factor
------------------------------------
   - 'Safe Load' was     556.54 MW    // considering safety factor
   - 'Unsafe Load' was   (81.3) MW    // need to shift
   - Load shifting to    62,096 homes // 81.3 MW shifted
------------------------------------
All Data
       output  demand    net_cap  s_net_cap  share_of_net   new_load
e_id                                                                
219    88.200   73.50  14.700000     5.8800      0.333333  27.115690
220    88.200   73.50  14.700000     5.8800      0.333333  27.115690
223    88.200   73.50  14.700000     5.8800      0.333333  27.115690
216   111.576  105.00   6.576097    -4.5816      0.056323  -4.

In [333]:
len('   - able to shift         41.7 MW    ')

38

In [46]:
df[(df.ts == 37) & (df.tracker == 0)]

,id,ts,host,tracker,output,demand,net_cap,net_cap_percent
288,-1293263167,37,0,0,122.641489,105.00,17.641489,0.143846
289,-728242845,37,0,0,38.028159,42.75,-4.721841,-0.124167
290,-1675485352,37,0,0,88.200000,73.50,14.700000,0.166667
291,-195991192,37,0,0,122.641489,105.00,17.641489,0.143846
292,-182766963,37,0,0,88.200000,73.50,14.700000,0.166667
293,-1719963416,37,0,0,122.641489,105.00,17.641489,0.143846
294,-348750876,37,0,0,88.200000,73.50,14.700000,0.166667
295,-23457213,37,0,0,37.360998,42.00,-4.639002,-0.124167


In [52]:
_ = df[(df.net_cap_percent < 0.125) & (df.tracker == 0) & (df.ts == 37)]
_ 

,id,ts,host,tracker,output,demand,net_cap,net_cap_percent
289,-728242845,37,0,0,38.028159,42.75,-4.721841,-0.124167
295,-23457213,37,0,0,37.360998,42.00,-4.639002,-0.124167


In [56]:
load_to_shift = sum(_.output * _.net_cap_percent)
load_to_shift

-9.360843566211205